<a href="https://colab.research.google.com/github/Ok3ks/Disso-COLD/blob/main/CNN_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install sklearn
%pip transformers
%pip install huggingface
%pip install torch 
%pip install pandas
%pip install sklearn|
%pip install datasets
%pip install transformers

In [4]:
from datasets import load_dataset, load_metric
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,get_scheduler
from torch.optim import AdamW
from spacy import tokenizer
from spacy.lang.en import English
import torch.nn as nn
from torch.utils.data import DataLoader

In [5]:
accuracy = load_metric('accuracy'); f1_score = load_metric('f1'); perplexity = load_metric('perplexity')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [6]:
def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return f1_score.compute(predictions, references = label)

In [24]:
from transformers import BertModel
#Using BERT Embeddings, i.e input ids

Bert_model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
max_input_length = 450

def bert_tokenizer(input):
  return tokenizer(input['text'], truncation=True, padding = True, max_length=max_input_length, return_tensors = 'pt')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
class CNN_BERT(nn.Module):

  #Params represents the parameters of a conv1d function with input, output, kernel size and stride.

  def __init__(self,model):
    super().__init__()
    self.conv = nn.Conv1d(768, 200, 5)
    self.conv = nn.Conv1d(200, 50, 3)
    self.pool = nn.MaxPool1d(kernel_size = 5) #Number of words it skips 
    self.relu = nn.ReLU()
    self.Linear_2 = nn.Linear(15,2)
    self.tokenizer = tokenizer
    self.model = model

  def forward(self,input):
    tokened = tokenizer(input['text'], truncation = True, max_length = max_input_length, return_tensors = 'pt')
    embeds = Bert_model(tokened['input_ids'], tokened['token_type_ids'])
    embeds = embeds[2][-2]
    out = self.conv(embeds)
    out = self.pool(out)
    out = self.relu(out)
    self.Linear = nn.Linear(out.shape[-1], 15)
    out = self.Linear(out)
    out = self.Linear_2(out)

In [9]:
imdb = load_dataset("imdb")
imdb = imdb.shuffle(seed = 42)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
load_dataset('imdb', split = "train")

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [11]:
#0 corresponds to negative review while 1 corresponds to positive reviews
label2text = {0:'negative', 1: 'positive'}

train_imdb = imdb['train']
test_imdb = imdb['test']
set(train_imdb['label'])

imdb_train_text = {i:x for i,x in enumerate(train_imdb['text'])}
imdb_test_text = {i:x for i,x in enumerate(test_imdb['text'])}

temp = []
imdb_labels_text_train = {i:temp.append(x) for i,x in zip(train_imdb['label'], train_imdb['text'])}

temp = []
imdb_labels_text_test = {i:temp.append(x) for i,x in zip(test_imdb['label'], test_imdb['text'])}

imdb_train_labels = {i:x for i,x in enumerate(train_imdb['label'])}
imdb_test_labels = {i:x for i,x in enumerate(test_imdb['label'])}

In [ ]:
pip install pytorch_lightning

In [26]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class load_data(pl.LightningDataModule):
  def __init__(self, batch_size: int = 32):
    super().__init__()
    self.train = load_dataset('imdb', split = "train").train_test_split(test_size = 0.2, seed = 42,  shuffle = True)
    self.test = load_dataset('imdb', split = "test")
    self.batch_size = batch_size
  
  def train_dataloader(self):
    return DataLoader(self.train['train'], batch_size = self.batch_size)
  def val_dataloader(self):
    return DataLoader(self.train['test'], batch_size = self.batch_size)
  def test_dataloader(self):
    return DataLoader(self.test, batch_size = self.batch_size)
  def predict_dataloader(self):
    return DataLoader(self.test, batch_size = self.batch_size)

In [15]:
import torch.nn.functional as F

In [30]:
class run(pl.LightningModule):
  """Use cross entropy loss in training"""

  def __init__(self):
    super().__init__()
    self.model = CNN_BERT(Bert_model) 

  def forward(self, text):
    probs = self.model(text)
    return probs

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

  def _common_step(self, batch, batch_idx):
    text, labels = batch
    labels = labels
    probs = self.forward(text)
    loss = F.cross_entropy(probs, labels)

    with torch.no_grad():
      preds = torch.round(probs.squeeze(1))
      num_correct = torch.sum(preds == labels.squeeze(1)).item() 
      num_total = labels.size(0)
      accuracy = num_correct / float(num_total)
    return loss, accuracy

  def training_step(self, train_batch, batch_idx):
    loss, acc = self._common_step(train_batch, batch_idx)
    self.log('train_loss', loss)
    self.log('train_acc', acc, prog_bar=True)
    return loss

  def validation_step(self, dev_batch, batch_idx):
    loss, acc = self._common_step(dev_batch, batch_idx)
    self.log('dev_loss', loss)
    self.log('dev_acc', acc, prog_bar=True)

  def test_step(self, test_batch, batch_idx):
    loss, acc = self._common_step(test_batch, batch_idx)
    self.log('test_loss', loss)
    self.log('test_acc', acc)

  def predict_step(self, batch, batch_idx):
    return self.forward(batch[0])

In [31]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
import torch

dm = load_data()
model = run()

checkpoint_callback = ModelCheckpoint(monitor = 'dev_loss')

torch.cuda.manual_seed_all(40)
trainer = Trainer(max_epochs = 20, gpus = 1, callbacks = [checkpoint_callback])
trainer.fit(model, dm)

results = trainer.test(model, dm, ckpt_path = "best")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | CNN_BERT | 109 M 
-----------------------------------
109 M     Trainable params
0         Non-trainable params
109

Sanity Checking: 0it [00:00, ?it/s]

TypeError: ignored

In [ ]:
import torch 
hidden_states = []; hidden_states_2 = []

with torch.no_grad():
  output = model(imdb_tokenized['input_ids'][:500],imdb_tokenized['token_type_ids'][:500])
  hidden_states.append(output[2][-2]) #Using output representation of token from penultimate layer
  hidden_states_2.append(output[1]) #Using representation of each CLS token